In [7]:
import pandas as pd

# Load the raw data
df = pd.read_csv('../data/raw/lsapp.tsv', sep='\t')

# Create interaction_id and session_id columns using the logic from your provided code
df['timestamp'] = pd.to_datetime(df['timestamp'])

df['interaction_id'] = (
    ((df.timestamp - df.timestamp.shift(1) > pd.Timedelta(1, 'm')) & (df.event_type == 'Opened'))
    | ~(df.app_name == df.app_name.shift(1))
    | ~(df.user_id == df.user_id.shift(1))
).cumsum()

df['session_id'] = (
    ((df.timestamp - df.timestamp.shift(1) > pd.Timedelta(5, 'm')) & (df.event_type == 'Opened'))
    | ~(df.user_id == df.user_id.shift(1))
).cumsum()

# Get ALL unique interaction_ids that exist in the data
all_interaction_ids = df['interaction_id'].unique()

# For each interaction, get the open and close times
df_start = df[df['event_type'] == 'Opened'].drop_duplicates(subset=['interaction_id'], keep='first')
df_end = df[df['event_type'] == 'Closed'].drop_duplicates(subset=['interaction_id'], keep='last')

# Create a complete DataFrame with ALL interaction_ids
df_complete = pd.DataFrame({'interaction_id': all_interaction_ids})

# Merge with start events (Opened)
df_start = df_start.set_index('interaction_id')
df_complete = df_complete.set_index('interaction_id')
df_complete = df_complete.join(df_start, how='left')

# Merge with end events (Closed)
df_end = df_end.set_index('interaction_id')
df_complete['close_time'] = df_end['timestamp']

# Duration in minutes - will be NaN for incomplete sessions
df_complete['duration'] = (df_complete['close_time'] - df_complete['timestamp']).dt.total_seconds() / 60.0

# Reset index to get interaction_id as a column
df_clean = df_complete.reset_index()

# Rename columns for clarity
df_clean = df_clean.rename(columns={'timestamp': 'open_time'})

# Add required columns

# is_weekend
df_clean['is_weekend'] = df_clean['open_time'].dt.weekday >= 5

# time_since_last_app (in minutes)
df_clean = df_clean.sort_values(['user_id', 'session_id', 'open_time'])
df_clean['time_since_last_app'] = df_clean.groupby(['user_id', 'session_id'])['open_time'].diff().dt.total_seconds() / 60.0

# hour_of_day
df_clean['hour_of_day'] = df_clean['open_time'].dt.hour

# time_since_session_start (in minutes)
df_clean['session_start_time'] = df_clean.groupby(['user_id', 'session_id'])['open_time'].transform('min')
df_clean['time_since_session_start'] = (df_clean['open_time'] - df_clean['session_start_time']).dt.total_seconds() / 60.0
df_clean = df_clean.drop(columns=['session_start_time'])

# day_0 ... day_6 (one-hot for day of week, where Monday=0)
for i in range(7):
    df_clean[f'day_{i}'] = (df_clean['open_time'].dt.weekday == i)

# Drop event_type and any columns not needed
columns_to_keep = [
    'user_id', 'session_id', 'interaction_id', 'app_name', 'open_time', 'duration',
    'is_weekend', 'time_since_last_app', 'hour_of_day', 'time_since_session_start'
]
# columns_to_keep = [
#     'user_id', 'session_id', 'interaction_id', 'app_name', 'open_time', 'duration',
#     'is_weekend', 'time_since_last_app', 'hour_of_day', 'time_since_session_start',
#     'day_0', 'day_1', 'day_2', 'day_3', 'day_4', 'day_5', 'day_6'
# ]
df_clean = df_clean[columns_to_keep]

# Save cleaned data
df_clean.to_csv('../data/cleaned/LSAPP_Processed.csv', index=False)

# Print summary of incomplete sessions
incomplete_count = df_clean['duration'].isna().sum()
total_count = len(df_clean)
print(f"Total interactions: {total_count}")
print(f"Incomplete sessions (null duration): {incomplete_count}")
print(f"Percentage incomplete: {incomplete_count/total_count:.2%}")

# Check for missing interaction_ids
expected_ids = set(range(1, df['interaction_id'].max() + 1))
actual_ids = set(df_clean['interaction_id'].dropna())
missing_ids = expected_ids - actual_ids
print(f"Missing interaction_ids: {sorted(missing_ids)}")

Total interactions: 599635
Incomplete sessions (null duration): 325388
Percentage incomplete: 54.26%
Missing interaction_ids: []
